In [3]:
import torch

# First, let's inspect the checkpoint structure
model_path = "/mnt/c/Users/JakeLee/Desktop/PFE/App/SkinVision-AI/backend/models/resnet50_model.pth"
checkpoint = torch.load(model_path, map_location='cpu')

print("Checkpoint keys:", checkpoint.keys())
print("\nSample model state dict keys:")
if 'model_state_dict' in checkpoint:
    model_keys = list(checkpoint['model_state_dict'].keys())
    for i, key in enumerate(model_keys[:10]):
        print(f"{i+1}. {key}")
    print(f"... and {len(model_keys)-10} more keys")
else:
    # Direct model state dict
    model_keys = list(checkpoint.keys())
    for i, key in enumerate(model_keys[:10]):
        print(f"{i+1}. {key}")
    print(f"... and {len(model_keys)-10} more keys")

Checkpoint keys: dict_keys(['epoch', 'model_state_dict', 'optimizer_state_dict', 'val_acc'])

Sample model state dict keys:
1. conv1.weight
2. bn1.weight
3. bn1.bias
4. bn1.running_mean
5. bn1.running_var
6. bn1.num_batches_tracked
7. layer1.0.conv1.weight
8. layer1.0.bn1.weight
9. layer1.0.bn1.bias
10. layer1.0.bn1.running_mean
... and 314 more keys


In [11]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime

# Define the correct model architecture that matches the saved checkpoint
class SkinLesionClassifier(nn.Module):
    def __init__(self, num_classes=40):  # Updated to 40 classes to match checkpoint
        super(SkinLesionClassifier, self).__init__()
        # Load ResNet50 and replace the final fc layer with custom layers
        resnet = models.resnet50(pretrained=False)
        
        # Copy all layers except the final fc layer
        self.conv1 = resnet.conv1
        self.bn1 = resnet.bn1
        self.relu = resnet.relu
        self.maxpool = resnet.maxpool
        self.layer1 = resnet.layer1
        self.layer2 = resnet.layer2
        self.layer3 = resnet.layer3
        self.layer4 = resnet.layer4
        self.avgpool = resnet.avgpool
        
        # Custom fc layers (matching the saved model structure exactly)
        # The saved model has fc.0, fc.3, fc.6 as Linear layers
        self.fc = nn.Sequential(
            nn.Linear(2048, 512),      # fc.0 - matches fc.0.weight, fc.0.bias
            nn.ReLU(inplace=True),     # fc.1 - ReLU activation
            nn.Dropout(0.5),           # fc.2 - Dropout
            nn.Linear(512, 256),       # fc.3 - matches fc.3.weight, fc.3.bias  
            nn.ReLU(inplace=True),     # fc.4 - ReLU activation
            nn.Dropout(0.3),           # fc.5 - Dropout
            nn.Linear(256, num_classes)# fc.6 - matches fc.6.weight, fc.6.bias
        )
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# Load the model
model_path = "/mnt/c/Users/JakeLee/Desktop/PFE/App/SkinVision-AI/backend/models/resnet50_model.pth"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load checkpoint
checkpoint = torch.load(model_path, map_location=device)

# Create model and load state dict
model = SkinLesionClassifier()
model.load_state_dict(checkpoint['model_state_dict'])
epoch = checkpoint.get('epoch', 'Unknown')
val_acc = checkpoint.get('val_acc', 'Unknown')

model.to(device)
model.eval()

print(f"Model loaded successfully on {device}")
print(f"Model was saved at epoch: {epoch}")
print(f"Validation accuracy: {val_acc:.4f}")

# Load class names from classes.txt file
def load_class_names():
    try:
        with open('/mnt/c/Users/JakeLee/Desktop/PFE/App/SkinVision-AI/classes.txt', 'r') as f:
            class_names = [line.strip() for line in f.readlines()]
        # Ensure we have exactly 40 classes
        if len(class_names) == 39:
            class_names.append("Unknown_Class_40")  # Add placeholder for missing class
            print("Warning: Only 39 classes found in classes.txt, added placeholder for 40th class")
        return class_names
    except FileNotFoundError:
        print("classes.txt not found, using default classes")
        # Create 40 default classes
        return [f"Class_{i+1}" for i in range(40)]

class_names = load_class_names()
print(f"Loaded {len(class_names)} class names")

# Define image transformations (adjust based on your training preprocessing)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Function to get available images
def get_available_images():
    image_dir = "/mnt/c/Users/JakeLee/Desktop/PFE/ChatGPT-TEST/Images for test"
    if not os.path.exists(image_dir):
        print(f"Directory not found: {image_dir}")
        return [], ""
    
    valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff')
    images = [f for f in os.listdir(image_dir) if f.lower().endswith(valid_extensions)]
    return images, image_dir

# Function to predict image and return results
def predict_image(image_path):
    try:
        # Load and preprocess image
        image = Image.open(image_path).convert('RGB')
        input_tensor = transform(image).unsqueeze(0).to(device)
        
        # Make prediction
        with torch.no_grad():
            outputs = model(input_tensor)
            probabilities = torch.nn.functional.softmax(outputs[0], dim=0)
            
        # Get top 5 predictions
        top5_probs, top5_indices = torch.topk(probabilities, 5)
        
        # Return results as lists
        predictions = []
        confidences = []
        
        for prob, idx in zip(top5_probs, top5_indices):
            predictions.append(class_names[idx.item()])
            confidences.append(prob.item())
        
        return predictions, confidences
        
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return ["Error"] * 5, [0.0] * 5

# Function to process all images and create Excel file
def process_all_images():
    images, image_dir = get_available_images()
    
    if not images:
        print("No images found in the directory!")
        return
    
    print(f"Processing {len(images)} images...")
    
    # Initialize results list
    results = []
    
    # Process each image
    for i, image_name in enumerate(images):
        print(f"Processing image {i+1}/{len(images)}: {image_name}")
        
        image_path = os.path.join(image_dir, image_name)
        predictions, confidences = predict_image(image_path)
        
        # Create row for this image
        row = {
            'Image_Name': image_name,
            'Prediction_1': predictions[0],
            'Confidence_1': confidences[0],
            'Prediction_2': predictions[1],
            'Confidence_2': confidences[1],
            'Prediction_3': predictions[2],
            'Confidence_3': confidences[2],
            'Prediction_4': predictions[3],
            'Confidence_4': confidences[3],
            'Prediction_5': predictions[4],
            'Confidence_5': confidences[4]
        }
        
        results.append(row)
    
    # Create DataFrame
    df = pd.DataFrame(results)
    
    # Generate filename with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    excel_filename = f"skin_lesion_predictions_{timestamp}.xlsx"
    excel_path = f"/mnt/c/Users/JakeLee/Desktop/PFE/App/SkinVision-AI/{excel_filename}"
    
    # Save to Excel
    df.to_excel(excel_path, index=False)
    
    print(f"\nProcessing complete!")
    print(f"Results saved to: {excel_filename}")
    print(f"Total images processed: {len(images)}")
    
    # Display summary statistics
    print("\nSummary:")
    top_predictions = df['Prediction_1'].value_counts().head(10)
    print("Top 10 most frequent predictions:")
    for pred, count in top_predictions.items():
        print(f"  {pred}: {count} images")
    
    return df

# Function to display sample predictions (optional)
def show_sample_predictions(df, num_samples=5):
    print(f"\nShowing top {num_samples} predictions:")
    for i in range(min(num_samples, len(df))):
        row = df.iloc[i]
        print(f"\nImage: {row['Image_Name']}")
        print(f"Top prediction: {row['Prediction_1']} ({row['Confidence_1']:.4f})")
        for j in range(2, 6):
            print(f"  {j}. {row[f'Prediction_{j}']} ({row[f'Confidence_{j}']:.4f})")

# Run the batch processing
results_df = process_all_images()

# Show sample results
if results_df is not None and len(results_df) > 0:
    show_sample_predictions(results_df)

/home/jl/miniconda3/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jl/miniconda3/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Model loaded successfully on cuda
Model was saved at epoch: 76
Validation accuracy: 70.9730
Loaded 40 class names
Processing 100 images...
Processing image 1/100: DERM_106035.jpg
Processing image 2/100: DERM_110135.jpg
Processing image 3/100: DERM_111637.jpg
Processing image 4/100: DERM_125826.jpg
Processing image 5/100: DERM_131011.jpg
Processing image 6/100: DERM_134644.jpg
Processing image 7/100: DERM_175311.jpg
Processing image 8/100: DERM_181085.jpg
Processing image 9/100: DERM_188685.jpg
Processing image 10/100: DERM_195022.jpg
Processing image 11/100: DERM_195672.jpg
Processing image 12/100: DERM_204464.jpg
Processing image 13/100: DERM_216816.jpg
Processing image 14/100: DERM_217371.jpg
Processing image 15/100: DERM_239326.jpg
Processing image 16/100: DERM_244584.jpg
Processing image 17/100: DERM_251874.jpg
Processing image 18/100: DERM_255498.jpg
Processing image 19/100: DERM_260248.jpg
Processing image 20/100: DERM_274993.jpg
Processing image 21/100: DERM_291844.jpg
Processin